In [3]:
import numpy as np
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import euclidean_distances
from collections import defaultdict
import spacy 

In [4]:
  
dataset= pd.read_json('sample_0_10000_books.json')

#dataset directly from 




In [5]:
print (len(dataset.keys()))

7454


# Dictionary Index

In [7]:
vectorizer = TfidfVectorizer(stop_words='english' ,
                     binary=False,
                     max_df=0.95, 
                     min_df=0.02,
                     norm = 'l2')

In [8]:
# create a list of reviews rather than creating a list of list 
#dictionary for each keys 
#need to create it in one list but is not creating in one list 


#key : title #value : string of all 10 reviews in one string 
onlyreviews = defaultdict(str) 
for keys in dataset.keys():
    #keys = keys.encode('UTF8')
    keyreview = ""
   
    datasets = dataset[keys]['review'] 
    #print(type(datasets))
   
    for i in datasets :  
        if type(i)!= float : 
         
            keyreview = keyreview + i[0].encode('UTF8')             
    keys = keys.encode('UTF8') 
    onlyreviews[keys] = keyreview
print(type(onlyreviews["A Book of Common Prayer"]) ) 


<type 'str'>


In [22]:
book_array = onlyreviews.keys()

book_to_index= {book : i for i, book in enumerate(book_array)}

index_to_book = {i : book for i, book in enumerate(book_array)}
review_vectors = vectorizer.fit_transform([onlyreviews[keys] for keys in onlyreviews]).toarray()

#Need to tokenize from the vectors

In [34]:
wordlist = vectorizer.get_feature_names()  
int_word = defaultdict(int) 
for i in range(len(wordlist)) : 
    int_word[i] = wordlist[i]    

def word_cloud(title) : 
    book_idx  = book_to_index[title]  
    vec = review_vectors[book_idx]   
    idxsortall = np.argsort(vec)[::-1][:50] 

    feature_list = [] 
    for i in idxsortall : 
        feature_list.append(int_word[i])
    
    adjective_list = []  
    noun_list = []  
    for i in feature_list : 
        doc = nlp(i) 
        for word in doc : 
            if word.pos_ == "ADJ" : 
                adjective_list.append(word) 
            elif word.pos_ == "NOUN" : 
                noun_list.append(word) 
            else : 
                continue  
    for i in adjective_list : 
        print(i) 
        
    print("nouns from here")
    for j in noun_list : 
        print(j)


In [35]:
book_to_index['Harry Potter and the Chamber of Secrets (Harry Potter, #2) (by) J.K. Rowling (published by) Scholastic']

1817

In [36]:
word_cloud('Harry Potter and the Chamber of Secrets (Harry Potter, #2) (by) J.K. Rowling (published by) Scholastic')

favorite
epic
entertaining
nouns from here
br
book
series
pictures
movie
words
https
www
review
scenes
christmas
audio
brother
feelings
com
reread
love
rest
ones
haven
list


# Cosine Similarity Matrix Using Book Title and similarity Matrix 


In [13]:
#similarity Matrix Creation 
def get_sim(book1, book2, input_doc_mat, book_to_index):
    """Returns a float giving the cosine similarity of 
       the two movie transcripts.
    
    Params: {book1: String,
             book2: String,
             input_doc_mat: Numpy Array,
             movie_to_index: Dict}
    Returns: Float (Cosine similarity of the two movie transcripts.)
    """
    # YOUR CODE HERE

    book1idx = book_to_index[book1] 
    book2idx = book_to_index[book2] 
    
    mat1 = input_doc_mat[book1idx] 
    mat2 = input_doc_mat[book2idx] 
    num = np.dot(mat1, mat2) 
    
    onenorm  = np.sqrt(np.sum(np.square(mat1))) 
    twonorm = np.sqrt(np.sum(np.square(mat2)))  
    
    denum = onenorm * twonorm 
     
    cosine = num/denum
    
    return cosine


In [14]:
def build_book_sims_cos(n_books,index_to_book,input_doc_mat,book_to_index,input_get_sim_method):
    """Returns a movie_sims matrix of size (num_movies,num_movies) where for (i,j):
        [i,j] should be the cosine similarity between the movie with index i and the movie with index j
        
    Note: What you set the diagonal to doesn't really matter, 
    but you should set it to 1 to indicate that all movies are trivially perfectly similar to themselves.
    
    Params: {n_book: Integer,
             index_to_book: String,
             input_doc_mat: Numpy Array,
             movie_to_index: Dict,
             input_get_sim_method: Function}
    Returns: Numpy Array 
    """
    # YOUR CODE HERE
    #hopefully this is okay! I guess 
    returnarray = np.zeros((n_books, n_books))
    for i in range(n_books) : 
        iname = index_to_book[i]  
        for j in range(i): 
            jname = index_to_book[j] 
            number = input_get_sim_method(iname, jname, input_doc_mat, book_to_index)
            returnarray[i][j] = number 
            returnarray[j][i] = number 
        returnarray[i][i] = 1.0 
    return returnarray

            

In [48]:
#create cosine matrix 
num_books = len(index_to_book)
books_list = dataset.keys() 
review_sims_cos = build_book_sims_cos(num_books, index_to_book, review_vectors, book_to_index, get_sim)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


In [382]:
#draw the matrix by the matlib 
assert type(book_sims_cos) == np.ndarray
assert book_sims_cos.shape == (num_books,num_books)


# Print top Ten books similar to the given book  reviews

In [383]:
#similarity measure print top ten that are related to one another 
#top ten similarity of the books related to the books that are given 


In [384]:
#import matplotlib.pyplot as plt

#plt.imshow(book_sims_cos , interpolation='none',
      #     cmap=plt.cm.Greys)
#plt.colorbar()

#plt.xticks(range(num_books),books_list , rotation=90)
#plt.yticks(range(num_books), books_list)
#plt.show()


In [15]:
def get_ranked_books(book, matrix):
    """
    Return sorted rankings (most to least similar) of movies as 
    a list of two-element tuples, where the first element is the 
    movie name and the second element is the similarity score
    
    Params: {book: String,
             matrix: np.ndarray}
    Returns: List<Tuple>
    """
    
    # Get movie index from movie name
    book_idx = book_to_index[book]
    
    # Get list of similarity scores for movie
    score_lst = matrix[book_idx]
    book_score_lst = [(index_to_book[i], s) for i,s in enumerate(score_lst)]
    
    # Do not account for movie itself in ranking
    book_score_lst = book_score_lst[:book_idx] + book_score_lst[book_idx+1:]
    
    # Sort rankings by score
    book_score_lst = sorted(book_score_lst, key=lambda x: -x[1])
    
    return book_score_lst


def print_top(book, matrix, sim_type, k=10):
    """
    Print the k most and least similar movies to 'star wars'
    
    Params: {book: String,
             matrix: np.ndarray,
             sim_type: String,
             k: Integer}
    Returns: None
    """
    
    book_score_lst = get_ranked_books(book, matrix)
    
    print("Top {} most similar books to {} [{}]".format(k, book , sim_type))
    print("======")
    for (book, score) in book_score_lst[:k]:
        print("%.3f %s" % (score, book))

    print()
    
    print("Top {} least similar books to {} [{}]".format(k, book , sim_type))
    print("======")
    for (book, score) in book_score_lst[-k:][::-1]:
        print("%.3f %s" % (score, book))

In [16]:
print_top('Thomas Jefferson ', book_sims_cos, 'cosine sim')

NameError: name 'book_sims_cos' is not defined

In [265]:
# Evaluation Check whether the books exist in the book dictionary 
#book 

In [389]:
book_array = onlyreviews.keys() 
print(book_array)
lstofbooks = [u'Child of the Prophecy (Sevenwaters, #3)',
  u'Gregor and the Marks of Secret (Underland Chronicles, #4)',
  u'Emperor Mage (Immortals, #3)',
  u"The Queen of Attolia (The Queen's Thief, #2)",
  u"The Wizard's Dilemma (Young Wizards, #5)",
  u'Awakening (Sweep, #5)',
  u'A Ring of Endless Light (Austin Family, #4)',
  u'A Muralha de Gelo (As Cr\xf3nicas de Gelo e Fogo #2)',
  u'Black Heart (Curse Workers, #3)',
  u'The Indigo Spell (Bloodlines, #3)',
  u'The Atlantis Complex (Artemis Fowl, #7)',
  u"The Demon's Covenant (The Demon's Lexicon, #2)",
  u'Syren (Septimus Heap, #5)',
  u'Haunted (The Mediator, #5)',
  u'The Farthest-Away Mountain',
  u'Dark Whispers (Unicorn Chronicles, #3)',
  u'Curse of the Blue Tattoo: Being an Account of the Misadventures of Jacky Faber, Midshipman and Fine Lady (Bloody Jack, #2)',
  u'The Subtle Knife (His Dark Materials, #2)']  
newlstofbooks = [] 

for k in lstofbooks :  
    newlstofbooks.append(k.encode('UTF8'))

    
for k in newlstofbooks : 
    if k in book_array : 
        print(True)
        
if 'Caesar (Masters of Rome, #5) (by) Colleen McCullough (published by) Avon' in book_array : 
    print(True)

     

['Caesar (Masters of Rome, #5) (by) Colleen McCullough (published by) Avon', 'El Mundo Perdido (Parque Jur\xc3\xa1sico, #2) (by) Michael Crichton (published by) DeBolsillo (Espa\xc3\xb1a)', 'Picnics in the Park: Moveable Feasts for Dining Alfresco (by) Connie McCole (published by) Hearst Communications', 'A Prayer for Owen Meany (by) Irving John (published by) Bloomsbury Publishing PLC', 'Introducing the Little Prince: Board Book Gift Set (by) Antoine de Saint-Exup\xc3\xa9ry (published by) Houghton Mifflin', "Papillions (by) Jacklyn E. Hungerland (published by) Barron's Educational Series", 'The Curious Incident of the Dog In the Night-time (by) Mark Haddon (published by) Red Fox', 'Fabeln. Text/ Kommentar. (Lernmaterialien) (by) Phaedrus (published by) Buchner', 'The Nature of Space and Time (by) Stephen Hawking (published by) Princeton University Press', 'Laura Ingalls Wilder: Pioneer and Author : Biographies for Young Readers (by) William Anderson (published by) Kipling Pr', 'A Guid

# Cosine Similarity Based on Description 


In [17]:
vectorizer = TfidfVectorizer(stop_words='english' ,
                     binary=False,
                     max_df=0.95, 
                     min_df=5,
                     norm = 'l2')

In [18]:
onlydescs = defaultdict(str) 
for k in dataset.keys() : 
    title = k.split('(by)')[0]
    if dataset[k]['description'] is not None : 
        onlydescs[title] = dataset[k]['description']



In [19]:
descs_vectors = vectorizer.fit_transform([onlydescs[keys] for keys in onlydescs]).toarray()

In [ ]:

num_books = len(index_to_book)
books_list = dataset.keys() 
book_sims_cos_short = build_book_sims_cos(num_books, index_to_book, descs_vectors, book_to_index, get_sim)[100:100] 

In [21]:
book_array = onlydescs.keys()

book_to_index= {book : i for i, book in enumerate(book_array)}

index_to_book = {i : book for i, book in enumerate(book_array)}




In [23]:
#change to json file i
import json 
with open('index_to_book.json', 'w') as fp:
    json.dump(index_to_book, fp)


In [24]:
with open('book_to_index.json','w') as fp: 
    json.dump(book_to_index, fp) 

In [23]:



book_sims_cos.tofile('descriptioncosine.csv'  ) 



In [25]:
print(type(book_sims_cos))

<type 'numpy.ndarray'>


In [29]:
np.savetxt('description_cosine.csv', book_sims_cos)

In [40]:
book_array[:100]

[u'Thomas Jefferson ',
 u"The Art of War: Sun Tzu's Classis in Plain English with Sun Pin's: The Art of Warfare ",
 u'What Every Fidelity Investor Needs to Know ',
 u'I Love Everybody (and Other Atrocious Lies) ',
 u'Journey to Nowhere: A New World Tragedy ',
 u"Melville's Moby Dick: An American Nekyia (Studies in Jungian Psychology by Jungian Analysts) ",
 u'Plata quemada ',
 u'Heaven and Other Poems ',
 u"Dragon's Eye ",
 u'The Leader In You: How to Win Friends, Influence People and Succeed in a Changing World ',
 u'If You Were There in 1776 ',
 u'Best Science Fiction Stories of H. G. Wells ',
 u'Pompeii: Monuments Past and Present ',
 u'Theater Shoes (Shoes, #4) ',
 u"The Tutor's First Love ",
 u'Point Deception (Cape Perdido, #1) ',
 u'One Day in History: July 4, 1776 ',
 u'The Edmund Fitzgerald: The Song of the Bell ',
 u'The Divine Comedy ',
 u'David Malouf ',
 u'Surfing the Himalayas: A Spiritual Adventure ',
 u'Zur Konzeption Des "Homme Supe\u0301rieur" Bei Stendhal Und Balzac:

In [45]:
print_top('Thomas Jefferson ', book_sims_cos, 'cosine sim description')

Top 10 most similar books to Thomas Jefferson  [cosine sim description]
0.436 Thomas Jefferson on Wine 
0.326 A Lesson Before Dying 
0.200 John Adams 
0.189 Early American Theatre from the Revolution to Thomas Jefferson: Into the Hands of the People 
0.120 The Founding Fathers and the Place of Religion in America 
0.112 The Portable John Adams 
0.102 The Unknown American Revolution: The Unruly Birth of Democracy and the Struggle to Create America 
0.099 Financial Founding Fathers: The Men Who Made America Rich 
0.079 Henry Adams and the Making of America 
0.071 The Tipping Point: How Little Things Can Make a Big Difference 
()
Top 10 least similar books to The Tipping Point: How Little Things Can Make a Big Difference  [cosine sim description]
0.000 Second Wind 
0.000 Formas breves 
0.000 The Peaceful Warrior: The Diary of Deborahs Armor Bearer, Israel, 1200 B.C. 
0.000 A Storm Without Rain: A Novel in Time 
0.000 Fluids and Electrolytes Made Incredibly Easy! 
0.000 Caught 'ya Again! 
